In [25]:
# data processing
import numpy as np
import pandas as pd
# pyplot
from matplotlib import pyplot as plt
# seaborn
import seaborn as sns
# ipywidgets
from ipywidgets import interact
# bokeh
from bokeh import plotting as bk
from bokeh.io import output_notebook, push_notebook
from bokeh.models import ColumnDataSource, Span, HoverTool, Slider, CustomJS
from bokeh.plotting import figure, output_file, show, curdoc
from bokeh.layouts import column, row
from bokeh.server.server import Server
from bokeh.application import Application
from bokeh.application.handlers.function import FunctionHandler
#utils
import os
from IPython.display import clear_output
output_notebook()

Loading BokehJS ...

In [26]:
# Compound utils
from compound import (
    get_compound_return,
    simulate_compound_return,
    build_dataframe,
    MONTHS_IN_YEAR,
    define_scenario,
    plot_scenario_bokeh
)

In [27]:
sns.set_style('darkgrid')
sns.color_palette("Paired")
sns.set_palette("Paired")

In [ ]:
step_amount = 5_000
max_amount = 20_000
initial_amounts = np.arange(step_amount,
                            max_amount + step_amount,
                            step_amount)
terms = [1, 3, 6, 12]
virtual_returns = np.arange(0.06, 0.12, 0.01).tolist()
annual_contributions = list(map(lambda x: 12 * x, range(50, 500, 50)))
inc_contributions = [0.01, 0.03]
years_of_investment = [30]#np.arange(30, 30 + 5, 5)
inflation_rates = [0.02, 0.03, 0.04]#np.arange(30, 30 + 5, 5)
retirement_years = [5, 10, 15, 20]
monthly_retirement_incomes = [
    1.0e2,
    2.0e2,
    3.0e2,
    4.0e2,
    5.0e2
    ]
#tax_rate = [0.25]
tax_rate = ["spain"]
retirement_contribution_rate = [0.1]

results = define_scenario(initial_amounts,
                          virtual_returns,
                          years_of_investment,
                          terms,
                          annual_contributions,
                          inc_contributions,
                          inflation_rates,
                          monthly_retirement_incomes,
                          retirement_years,
                          tax_rate
                          )

df = build_dataframe(results)
df = df[df["stable_yield"]]
df

1. Server-side solution to call Python callback in Bokeh

https://stackoverflow.com/questions/61695005/updating-a-graph-using-a-slider-with-python-and-bokeh

1. Grid lines

https://stackoverflow.com/questions/36244811/how-do-i-remove-grid-lines-from-a-bokeh-plot

1. Visual styles

https://docs.bokeh.org/en/2.4.1/docs/user_guide/styling.html

In [108]:
def make_page(doc, w=600, h=500):
    def common_callback_slider(attr, old, new):
        # Calculate the total balance and earnings using the updated function
        total_balance, info = simulate_compound_return(
            principal=slider_initial_amount.value,
            annual_roi=slider_annual_roi.value,
            compounding_frequency=slider_compound_frequency.value,
            annual_contribution=slider_annual_contribution.value,
            inc_contribution_rate=slider_contribution_inc.value,
            investment_duration=slider_duration_years.value,
            retirement_at=slider_extraction_start.value,
            monthly_retirement_income=slider_monthly_extraction.value,
            inflation_rate=slider_inflation_rate.value,
            tax_rate=slider_tax_rate.value,
            return_time_yields=True
        )
        source_e.data = dict(term=np.arange(len(info["earnings"])),
                             earning=info["earnings"]
                            )
        source_b.data = dict(month=np.arange(len(info["balances"])),
                             year=np.arange(len(info["balances"])) // 12,
                             balance=info["balances"]
                            )
        push_notebook()
        _, info = simulate_compound_return(
            principal=slider_initial_amount.value,
            annual_roi=slider_annual_roi.value,
            compounding_frequency=slider_compound_frequency.value,
            annual_contribution=slider_annual_contribution.value,
            inc_contribution_rate=slider_contribution_inc.value,
            investment_duration=slider_duration_years.value,
            retirement_at=slider_extraction_start.value,
            monthly_retirement_income=slider_monthly_extraction.value,
            inflation_rate=slider_inflation_rate.value,
            tax_rate=slider_tax_rate.value,
            return_time_yields=True
        )
    earnings = info["earnings"]
    balances = info["balances"]

    plot_earnings = bk.figure(width=w,
                              height=h,
                              title='Evolution of yields',
                              #tools='hover'
                              )
    # Annotations
    plot_earnings.xaxis.axis_label = "yield-term"
    plot_earnings.yaxis.axis_label = "earning (€)"
    # Grids
    plot_earnings.xgrid.grid_line_color = None
    # Background color
    plot_earnings.background_fill_color = "cyan"
    plot_earnings.background_fill_alpha = 0.1
    # Border color
    plot_earnings.border_fill_color = "gray"
    plot_earnings.border_fill_alpha = 0.2
    
    ## Earnings plot
    source_e = ColumnDataSource(
        data=dict(term=np.arange(len(earnings)),
                  earning=earnings
                 )
    )

    curve_e = plot_earnings.line(x='term',
                          y='earning',
                          source=source_e)

    # zero-earnings baseline
    hline_e = Span(location=earnings[0],
                   dimension='width',
                   line_color='red',
                   line_width=1,
                   line_dash='dashed')

    #plot.renderers.extend([curve, hline])

    # hover tools for earning curve
    hover_e = HoverTool(
        tooltips=[
            ('yield-term',  '@term'),
            ('earning-yield', '€ @earning{%0.2f}'), # use @{ } for field names with spaces
        ],

        formatters={
            '@yield-term': 'printf', # use default 'numeral' formatter for other fields
            '@earning': 'printf',   # use 'printf' formatter for '@{adj close}' field
        },
        # display a tooltip whenever the cursor is vertically in line with a glyph
        mode='vline'
    )

    ## Balances plot
    plot_balances = bk.figure(width=w,
                              height=h,
                              title='Evolution of balance',
                              #tools='hover'
                             )
    # Annotations
    plot_balances.xaxis.axis_label = "year"
    plot_balances.yaxis.axis_label = "balance (€)"
    # Grids
    plot_balances.xgrid.grid_line_color = None
    # Background
    plot_balances.background_fill_color = "cyan"
    plot_balances.background_fill_alpha = 0.1
    
    # Border color
    plot_balances.border_fill_color = "gray"
    plot_balances.border_fill_alpha = 0.2
    
    source_b = ColumnDataSource(
        data=dict(month=np.arange(len(balances)),
                  year=np.arange(len(balances)) // 12,
                  balance=balances
                  )

        )
    curve_b = plot_balances.line(x='year',
                                 y='balance',
                                 source=source_b)

    # zero-balance baseline
    hline_b = Span(location=earnings[0],
                   dimension='width',
                   line_color='red',
                   line_width=1,
                   line_dash='dashed')
    
    # Sliders
    slider_initial_amount = Slider(start=0.0, 
                                   end=100_000.0, 
                                   value=10_000.0, 
                                   step=200.0, 
                                   title="Initial amount"
                                  )
    slider_initial_amount.on_change("value", common_callback_slider)
    slider_annual_roi = Slider(start=0.0, 
                               end=1.0, 
                               value=0.06, 
                               step=0.01, 
                               title="Roi"
                              )
    slider_annual_roi.on_change("value", common_callback_slider)
    slider_compound_frequency = Slider(start=1, 
                                       end=12, 
                                       value=12, 
                                       step=1, 
                                       title="Compound frequency"
                                      )
    slider_compound_frequency.on_change("value", common_callback_slider)
    slider_annual_contribution = Slider(start=0.0, 
                                        end=10_000.0, 
                                        value=1200.0, 
                                        step=50.0, 
                                        title="Annual contribution"
                                       )
    slider_annual_contribution.on_change("value", common_callback_slider)
    slider_contribution_inc = Slider(start=0.0, 
                                     end=0.1, 
                                     value=0.01, 
                                     step=0.01, 
                                     title="Contribution increment"
                                    )
    slider_contribution_inc.on_change("value", common_callback_slider)
    slider_duration_years = Slider(start=1, 
                                   end=100, 
                                   value=60, 
                                   step=1, 
                                   title="Duration years"
                                  )
    slider_duration_years.on_change("value", common_callback_slider)
    slider_extraction_start = Slider(start=1, 
                                     end=100, 
                                     value=30, 
                                     step=1, 
                                     title="Year extraction start"
                                    )
    slider_extraction_start.on_change("value", common_callback_slider)
    slider_monthly_extraction = Slider(start=0.0, 
                                       end=10_000.0, 
                                       value=1_000.0, 
                                       step=50.0, 
                                       title="Monthly extraction"
                                      )
    slider_monthly_extraction.on_change("value", common_callback_slider)
    slider_inflation_rate = Slider(start=0.0, 
                                   end=0.5, 
                                   value=0.02, 
                                   step=0.005, 
                                   title="Inflation rate"
                                  )
    slider_inflation_rate.on_change("value", common_callback_slider)
    slider_tax_rate = Slider(start=0.0, 
                             end=1.0, 
                             value=0.2, 
                             step=0.005, 
                             title="Tax rate"
                            )
    slider_tax_rate.on_change("value", common_callback_slider)
    
    # hover tools for balance curve
    hover_b = HoverTool(
        tooltips=[
            ('month',  '@month (year @year)'),
            ('balance', '€ @balance{%0.2f}'), # use @{ } for field names with spaces
        ],

        formatters={
            '@year': 'printf', # use default 'numeral' formatter for other fields
            '@balance': 'printf',   # use 'printf' formatter for '@{adj close}' field
        },
        # display a tooltip whenever the cursor is vertically in line with a glyph
        mode='vline'
    )
    
    plot_earnings.add_layout(hline_e)
    plot_earnings.add_tools(hover_e)
    plot_balances.add_layout(hline_b)
    plot_balances.add_tools(hover_b)
    plot_row = row(plot_earnings, plot_balances)
    
    slider_column = column(slider_initial_amount,
                           slider_annual_roi,
                           slider_annual_contribution,
                           slider_compound_frequency,
                           slider_duration_years,
                           slider_contribution_inc,
                           slider_extraction_start,
                           slider_inflation_rate,
                           slider_monthly_extraction,
                           slider_tax_rate,
                           width=300
                          )
    layout = row(plot_row, slider_column)
    doc.add_root(row(plot_row, slider_column))
    return layout


In [109]:
try:
    server.stop()
except:
    pass
# creating the application and running the server local, (http://localhost:5000), port 5000 can be changed
apps = {'/': Application(FunctionHandler(make_page))}
server = Server(apps, port=8887)
server.start()
output_notebook()

Loading BokehJS ...

In [111]:
import bokeh.sampledata
bokeh.sampledata.download()

Using data directory: C:\Users\marsu\.bokeh\data
Skipping 'CGM.csv' (checksum match)
Skipping 'US_Counties.zip' (checksum match)
Skipping 'us_cities.json' (checksum match)
Skipping 'unemployment09.csv' (checksum match)
Skipping 'AAPL.csv' (checksum match)
Skipping 'FB.csv' (checksum match)
Skipping 'GOOG.csv' (checksum match)
Skipping 'IBM.csv' (checksum match)
Skipping 'MSFT.csv' (checksum match)
Skipping 'WPP2012_SA_DB03_POPULATION_QUINQUENNIAL.zip' (checksum match)
Skipping 'gapminder_fertility.csv' (checksum match)
Skipping 'gapminder_population.csv' (checksum match)
Skipping 'gapminder_life_expectancy.csv' (checksum match)
Skipping 'gapminder_regions.csv' (checksum match)
Skipping 'world_cities.zip' (checksum match)
Skipping 'airports.json' (checksum match)
Skipping 'movies.db.zip' (checksum match)
Skipping 'airports.csv' (checksum match)
Skipping 'routes.csv' (checksum match)
Skipping 'haarcascade_frontalface_default.xml' (checksum match)
Skipping 'SampleSuperstore.csv.zip' (chec

C:\Users\marsu\AppData\Local\Temp\ipykernel_17692\4020709738.py:24: UserWarning: Cannot find a last shown plot to update. Call output_notebook() and show(..., notebook_handle=True) before push_notebook()
  push_notebook()
